In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import weightedtau, kendalltau
from tqdm import tqdm

root_dir = "../"
sys.path.append(root_dir)
import configs
import datasets
from ibydmt.utils.config import get_config
from ibydmt.utils.data import get_dataset
from ibydmt.utils.result import TestingResults
from ibydmt.tester import sweep

config_name = "imagenette"
results_kw = {"testing.kernel_scale": 0.9, "testing.tau_max": 400}

config = get_config(config_name)
backbone_configs = sweep(config, sweep_keys=["data.backbone"])
dataset = get_dataset(config)
classes = dataset.classes

sns.set_style()
sns.set_context("paper")

In [ ]:
results = {}
for i, backbone_config in enumerate(tqdm(backbone_configs)):
    backbone = backbone_config.data.backbone
    try:
        backbone_results = TestingResults.load(
            backbone_config, "global", "dataset", results_kw=results_kw
        )
    except FileNotFoundError:
        continue

    results[backbone] = {}
    for class_name in classes:
        (
            sorted_idx,
            sorted_concepts,
            sorted_rejected,
            sorted_tau,
        ) = backbone_results.sort(class_name, fdr_control=True, with_importance=False)
        results[backbone][class_name] = {
            "concepts": sorted_concepts,
            "rejected": sorted_rejected,
            "tau": sorted_tau,
        }

In [ ]:
tau = np.zeros((len(classes), len(backbone_configs), len(backbone_configs)))

for i, (backbone1, results1) in enumerate(results.items()):
    for j, (backbone2, results2) in enumerate(results.items()):
        for class_idx, class_name in enumerate(classes):
            concepts1 = results1[class_name]["concepts"]
            concepts2 = results2[class_name]["concepts"]
            assert set(concepts1) == set(concepts2)

            idx1 = np.arange(len(concepts1))
            idx2 = np.array([concepts1.index(concept) for concept in concepts2])
            tau[class_idx, i, j] = weightedtau(idx1, idx2, rank=False).statistic

tau_mu = np.mean(tau, axis=0)
tau_std = np.std(tau, axis=0)
annot = np.array(
    [f"{mu:.2f}\n(±{std:.2f})" for mu, std in zip(tau_mu.flatten(), tau_std.flatten())]
).reshape(tau_mu.shape)

_, ax = plt.subplots(figsize=(5, 5))
sns.heatmap(
    tau_mu,
    vmin=-1.0,
    vmax=1.0,
    ax=ax,
    annot=annot,
    cmap="mako",
    fmt="",
    linecolor="black",
    linewidths=0.5,
    cbar_kws={"label": "Weighted Kendall's tau"},
    annot_kws={"fontsize": 7},
)
ax.axis("on")
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.set_xticklabels(list(results.keys()), rotation=45, ha="left")
ax.set_yticklabels(list(results.keys()), rotation=0)
plt.show()